# SENTIMENT ANALYSIS WITH SPARK ML

# Spark ML Main Concepts

The Spark Machine learning API in the **spark.ml** package is based on DataFrames, there is also another Spark Machine learning API based on RDDs in the **spark.mllib** package, but as of Spark 2.0, the RDD-based API has entered maintenance mode. The primary Machine Learning API for Spark is now the DataFrame-based API.

Main concepts of Spark ML:

- **Transformer**: transforms one DataFrame into another DataFrame

- **Estimator**: eg. a learning algorithm that trains on a DataFrame and produces a Model

- **Pipeline**: chains Transformers and Estimators to produce a Model

- **Evaluator**: measures how well a fitted Model does on held-out test data


# Amazon product data
We will use a [dataset](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz)[1] that contains 8.9M book reviews from Amazon, spanning May 1996 - July 2014.

Dataset characteristics:
- Number of reviews: 8.9M
- Size: 8.8GB (uncompressed)
- HDFS blocks: 70 (each with 3 replicas)


[1] Image-based recommendations on styles and substitutes
J. McAuley, C. Targett, J. Shi, A. van den Hengel
SIGIR, 2015
http://jmcauley.ucsd.edu/data/amazon/

# Load Data

For testing during the lab you can use a reduced dataset:

In [ ]:
%%time
raw_reviews = spark.read.json('/tmp/reviews_Books_5_small.json')

In [2]:
raw_reviews.count()

10000

This would be the full dataset (the times below were obtained with this dataset):

In [1]:
%%time
#raw_reviews = spark.read.json('data/amazon/reviews_Books_5.json')

CPU times: user 1 µs, sys: 2 µs, total: 3 µs
Wall time: 5.96 µs


In [2]:
%%time
all_reviews = raw_reviews.select('reviewText', 'overall')
all_reviews.cache()
all_reviews.show(2)

+--------------------+-------+
|          reviewText|overall|
+--------------------+-------+
|Spiritually and m...|    5.0|
|This is one my mu...|    5.0|
+--------------------+-------+
only showing top 2 rows

CPU times: user 3.91 ms, sys: 935 µs, total: 4.84 ms
Wall time: 3.77 s


# Prepare data
We will avoid neutral reviews by keeping only reviews with 1 or 5 stars overall score.
We will also filter out the reviews that contain no text.

In [3]:
nonneutral_reviews = all_reviews.filter(
    (all_reviews.overall == 1.0) | (all_reviews.overall == 5.0))
reviews = nonneutral_reviews.filter(all_reviews.reviewText != '')

In [4]:
reviews.cache()
all_reviews.unpersist()

DataFrame[reviewText: string, overall: double]

# Split Data

In [5]:
trainingData, testData = reviews.randomSplit([0.8, 0.2])

# Generate Pipeline
![pipeline](http://hadoop.cesga.es/files/sentiment_analysis/pipeline.jpg)

## Binarizer
A transformer to convert numerical features to binary (0/1) features

In [6]:
from pyspark.ml.feature import Binarizer

binarizer = Binarizer(threshold=2.5, inputCol='overall', outputCol='label')

## Tokenizer
A transformer that converts the input string to lowercase and then splits it by white spaces.

In [7]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")

## StopWordsRemover
A transformer that filters out stop words from input. Note: null values from input array are preserved unless adding null to stopWords explicitly.

In [8]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")

## HashingTF
A Transformer that converts a sequence of words into a fixed-length feature Vector. It maps a sequence of terms to their term frequencies using a hashing function.

In [9]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol="features")

# Estimator
## LogisticRegression

In [10]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Pipeline

In [11]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[binarizer, tokenizer, remover, hashingTF, lr])

In [12]:
%%time
pipeLineModel = pipeline.fit(trainingData)

CPU times: user 37 ms, sys: 13.2 ms, total: 50.2 ms
Wall time: 58.1 s


# Evaluation

In [13]:
%%time
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()

predictions = pipeLineModel.transform(testData)

aur = evaluator.evaluate(predictions)

print 'Area under ROC: ', aur

Area under ROC:  0.967783441159
CPU times: user 31.8 ms, sys: 4.17 ms, total: 36 ms
Wall time: 16.7 s


# Hyperparameter Tuning

In [14]:
%%time
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
param_grid = ParamGridBuilder() \
            .addGrid(hashingTF.numFeatures, [10000, 100000]) \
            .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
            .addGrid(lr.maxIter, [10, 20]) \
            .build()
            
cv = (CrossValidator()
      .setEstimator(pipeline)
      .setEvaluator(evaluator)
      .setEstimatorParamMaps(param_grid)
      .setNumFolds(3))

cv_model = cv.fit(trainingData)

CPU times: user 3.9 s, sys: 1.11 s, total: 5.01 s
Wall time: 12min 49s


In [15]:
%%time
new_predictions = cv_model.transform(testData)
new_aur = evaluator.evaluate(new_predictions)
print 'Area under ROC: ', new_aur

Area under ROC:  0.96977328792
CPU times: user 28.3 ms, sys: 8.81 ms, total: 37.1 ms
Wall time: 5.12 s
